# Calculating E2FCA Score for PHC, Secondary, Tertiary Hospitals for whole Shimane region by cho-cho aza level

# First, prepare all datasets in appropriate format

### Copy all datasets to a project folder

In [ ]:
import arcpy
from arcpy import env
try:
    
    env.workspace = "C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/HealthPlaceNew.gdb"
    ### Set a path for created data will ve stored
    outpath = "C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/HealthPlaceNew.gdb"
    # Export data to new path
    arcpy.FeatureClassToFeatureClass_conversion('PHCNaika_Chugoku',outpath,"PHCNaika_Chugoku")
    arcpy.FeatureClassToFeatureClass_conversion('Hos2Naika_Chugoku',outpath,"Hos2Naika_Chugoku")
    arcpy.FeatureClassToFeatureClass_conversion('Hos3Naika_Chugoku',outpath,"Hos3Naika_Chugoku")
    arcpy.FeatureClassToFeatureClass_conversion('KIHON_Chugoku22',outpath,"KIHON_Chugoku2010")
    arcpy.FeatureClassToFeatureClass_conversion("平成27年 国勢調査 町丁字等別集計",outpath,"Census2015Pop")

except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))    


### Create a Census polygon data and Census centroid point data

In [1]:
import arcpy
from arcpy import env
try:
    env.workspace = "C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/HealthPlaceNew.gdb"
    
    ### Make a centroid point feature of census data
    arcpy.Clip_analysis('Census2015Pop', 'LandNoLakeForest',"Census2015ResArea")
    # Create centroid point feature
    arcpy.FeatureToPoint_management("Census2015ResArea", "C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/HealthPlaceNew.gdb/Census2015Res_Centroid", "CENTROID")

except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

### Get a residential area size to compute an accurate population density

In [2]:
import arcpy
from arcpy import env
try:
    env.workspace = "C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/HealthPlaceNew.gdb"

    ### Calculate the size of each polygon (Residential area)
    # Set local variables
    inTable = "Census2015ResArea"
    fieldName = "AreaRes"
    expression = "!shape.area@SQUAREKILOMETERS!"
    # Execute AddField
    arcpy.AddField_management(inTable, fieldName, "DOUBLE")
    # Execute CalculateField - Get a area size in square kilometers
    arcpy.CalculateField_management(inTable, fieldName, expression, "PYTHON_9.3")

except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

### Join the field (residential area) back to Census Polygon data

In [5]:
import arcpy
from arcpy import env
import os
try:
    # Set environment settings
    env.workspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\HealthPlaceNew.gdb"
    
    # Set the local parameters
    inWorkspace1 = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\HealthPlaceNew.gdb"
    inFeatures = os.path.join(inWorkspace1,"Census2015PopNoLake")
    inWorkspace2 = r"C:\Users\kenta\Documents\HDDFiles\GISData_Import\Esri公共2016中国\公共地図\PUBLIC_MAP.gdb"
    eraseFeature = os.path.join(inWorkspace2,"WATER_LAKE") #Lake from Kokyo 2016
    joinField1 = "OBJECTID"
    joinTable = os.path.join(inWorkspace1,"Census2015ResArea")
    joinField2 = "OBJECTID"
    fieldList = ["AreaRes"] 
    outFeatures = os.path.join(inWorkspace1,"Census2015Final")
    
    # First create a Census Polygon data without Lake - ***This is for join, visualize, and final dataset
    arcpy.Erase_analysis('Census2015Pop', eraseFeature, inFeatures)
    # Join two feature classes by the OBJECTID field and only carry 
    # over the AreaRes field
    arcpy.JoinField_management(inFeatures, joinField1, joinTable, joinField2, fieldList)
    arcpy.CopyFeatures_management(inFeatures, outFeatures)

except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

### Calculate Population density

In [6]:
import arcpy
from arcpy import env
try:
    env.workspace = "C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/HealthPlaceNew.gdb"

    ### Calculate the size of each polygon (Residential area)
    # Set local variables
    inTable = "Census2015Final"
    fieldName = "PopDen"
    expression = "!TOTPOP_H27!/!AreaRes!"
    # Execute AddField
    arcpy.AddField_management(inTable, fieldName, "DOUBLE")
    # Execute CalculateField - Get a area size in square kilometers
    arcpy.CalculateField_management(inTable, fieldName, expression, "PYTHON_9.3")

except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

# Create network service layer for PHC

### PHC

In [ ]:
# This works!!
# PHC 45 facilities not created - 3 in Shimane
# Hos2 4 facilities not created - 1 in Shimane
# Hos3 2 facilities not created - Not in catchment (out of Shimane)

In [ ]:
import arcpy
from arcpy import env
import os
try:
    # First Create new gdb in the folder of HDD
    out_folder_path = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew" 
    out_name1 = "E2FCAStep1Rural.gdb"
    out_name2 = "E2FCAStep2Rural.gdb"
    out_name3 = "NetworkServiceRural.gdb"
    # Execute CreateFileGDB
    arcpy.CreateFileGDB_management(out_folder_path, out_name1)
    arcpy.CreateFileGDB_management(out_folder_path, out_name2)
    arcpy.CreateFileGDB_management(out_folder_path, out_name3)
    #Set environment settings
    output_dir = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew"
    #The NA layer's data will be saved to the workspace specified here
    env.workspace = os.path.join(output_dir, "NetworkServiceRural.gdb")
    env.overwriteOutput = True
    
    #Set local variables
    input_gdb1 = r"C:/Users/kenta/Documents/HDDFiles/GISData_Import/Esri道路2014中国/道路網_鳥取県_島根県_岡山県_広島県_山口県版/ADF2014.gdb"
    network = os.path.join(input_gdb1, "ADF","ADF_ND")
    layer_name = "PHCCover"
    travel_mode = "Driving Time"
    input_gdb2 = "C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/HealthPlaceNew.gdb"
    facilities = os.path.join(input_gdb2,"PHCNaika_Chugoku")
    output_layer_file = os.path.join(output_dir, layer_name + ".lyrx")
  
    
    #Create a new service area layer. We wish to generate the service area
    #polygons as rings, so that we can easily visualize the coverage for any
    #given location. We also want overlapping polygons so we can determine the
    #number of fire stations that cover a given location. We will specify these
    #options while creating the new service area layer.
    result_object = arcpy.na.MakeServiceAreaAnalysisLayer(network, layer_name,
                                    travel_mode, "FROM_FACILITIES", [10, 20, 30, 60],
                                    time_zone="LOCAL_TIME_AT_LOCATIONS",
                                    polygon_detail="STANDARD",
                                    geometry_at_overlaps="OVERLAP",
                                    geometry_at_cutoffs="RINGS")

    #Get the layer object from the result object. The service layer can now be
    #referenced using the layer object.
    layer_object = result_object.getOutput(0)

    #Get the names of all the sublayers within the service area layer.
    sublayer_names = arcpy.na.GetNAClassNames(layer_object)
    #Stores the layer names that we will use later
    facilities_layer_name = sublayer_names["Facilities"]

    #Load the fire stations as facilities using default field mappings and
    #default search tolerance
    arcpy.na.AddLocations(layer_object, facilities_layer_name, facilities, "", "")

    #Solve the service area layer
    arcpy.na.Solve(layer_object)

    #Save the solved service area layer as a layer file on disk
    layer_object.saveACopy(output_layer_file)

    print("Script completed successfully")

except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

### Hos2

In [ ]:
import arcpy
from arcpy import env
import os
try:
    #Set environment settings
    output_dir = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew"
    #The NA layer's data will be saved to the workspace specified here
    env.workspace = os.path.join(output_dir, "NetworkServiceRural.gdb")
    env.overwriteOutput = True
    
    #Set local variables
    input_gdb1 = r"C:/Users/kenta/Documents/HDDFiles/GISData_Import/Esri道路2014中国/道路網_鳥取県_島根県_岡山県_広島県_山口県版/ADF2014.gdb"
    network = os.path.join(input_gdb1, "ADF","ADF_ND")
    layer_name = "Hos2Cover"
    travel_mode = "Driving Time"
    input_gdb2 = "C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/HealthPlaceNew.gdb"
    facilities = os.path.join(input_gdb2,"Hos2Naika_Chugoku")
    output_layer_file = os.path.join(output_dir, layer_name + ".lyrx")
  
    
    #Create a new service area layer. We wish to generate the service area
    #polygons as rings, so that we can easily visualize the coverage for any
    #given location. We also want overlapping polygons so we can determine the
    #number of fire stations that cover a given location. We will specify these
    #options while creating the new service area layer.
    result_object = arcpy.na.MakeServiceAreaAnalysisLayer(network, layer_name,
                                    travel_mode, "FROM_FACILITIES", [10, 20, 30, 60],
                                    time_zone="LOCAL_TIME_AT_LOCATIONS",
                                    polygon_detail="STANDARD",
                                    geometry_at_overlaps="OVERLAP",
                                    geometry_at_cutoffs="RINGS")

    #Get the layer object from the result object. The service layer can now be
    #referenced using the layer object.
    layer_object = result_object.getOutput(0)

    #Get the names of all the sublayers within the service area layer.
    sublayer_names = arcpy.na.GetNAClassNames(layer_object)
    #Stores the layer names that we will use later
    facilities_layer_name = sublayer_names["Facilities"]

    #Load the fire stations as facilities using default field mappings and
    #default search tolerance
    arcpy.na.AddLocations(layer_object, facilities_layer_name, facilities, "", "")

    #Solve the service area layer
    arcpy.na.Solve(layer_object)

    #Save the solved service area layer as a layer file on disk
    layer_object.saveACopy(output_layer_file)

    print("Script completed successfully")

except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

### Hos3

In [ ]:
import arcpy
from arcpy import env
import os
try:
    #Set environment settings
    output_dir = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew"
    #The NA layer's data will be saved to the workspace specified here
    env.workspace = os.path.join(output_dir, "NetworkServiceRural.gdb")
    env.overwriteOutput = True
    
    #Set local variables
    input_gdb1 = r"C:/Users/kenta/Documents/HDDFiles/GISData_Import/Esri道路2014中国/道路網_鳥取県_島根県_岡山県_広島県_山口県版/ADF2014.gdb"
    network = os.path.join(input_gdb1, "ADF","ADF_ND")
    layer_name = "Hos3Cover"
    travel_mode = "Driving Time"
    input_gdb2 = "C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/HealthPlaceNew.gdb"
    facilities = os.path.join(input_gdb2,"Hos3Naika_Chugoku")
    output_layer_file = os.path.join(output_dir, layer_name + ".lyrx")
  
    
    #Create a new service area layer. We wish to generate the service area
    #polygons as rings, so that we can easily visualize the coverage for any
    #given location. We also want overlapping polygons so we can determine the
    #number of fire stations that cover a given location. We will specify these
    #options while creating the new service area layer.
    result_object = arcpy.na.MakeServiceAreaAnalysisLayer(network, layer_name,
                                    travel_mode, "FROM_FACILITIES", [10, 20, 30, 60],
                                    time_zone="LOCAL_TIME_AT_LOCATIONS",
                                    polygon_detail="STANDARD",
                                    geometry_at_overlaps="OVERLAP",
                                    geometry_at_cutoffs="RINGS")

    #Get the layer object from the result object. The service layer can now be
    #referenced using the layer object.
    layer_object = result_object.getOutput(0)

    #Get the names of all the sublayers within the service area layer.
    sublayer_names = arcpy.na.GetNAClassNames(layer_object)
    #Stores the layer names that we will use later
    facilities_layer_name = sublayer_names["Facilities"]

    #Load the fire stations as facilities using default field mappings and
    #default search tolerance
    arcpy.na.AddLocations(layer_object, facilities_layer_name, facilities, "", "")

    #Solve the service area layer
    arcpy.na.Solve(layer_object)

    #Save the solved service area layer as a layer file on disk
    layer_object.saveACopy(output_layer_file)

    print("Script completed successfully")

except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

# Spatial join with Population data

### PHC

In [1]:
### Import system modules
import arcpy
import os

try:
    
    # Set local variables
    inWorkspace1 = r"C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/NetworkServiceRural.gdb"
    inWorkspace2 = r"C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/HealthPlaceNew.gdb"
    outWorkspace = r"C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/E2FCAStep1Rural.gdb"
    
    # Want to join PHCCover polygon to KIHON_Chugoku2010 and calculate
    # the total number of population that service area (10, 20, 30, 60 min) can cover
    targetFeatures = os.path.join(inWorkspace1, "SAPolygons1")
    joinFeatures = os.path.join(inWorkspace2, "KIHON_Chugoku2010")
    
    # Output will be the target features: PHCCover Polygon with a sum of population field 
    outfc = os.path.join(outWorkspace, "PHCCoverPop")
    
    # Create a new fieldmappings and add the two input feature classes.
    fieldmappings = arcpy.FieldMappings()
    fieldmappings.addTable(targetFeatures)
    fieldmappings.addTable(joinFeatures)
    
    # First get the POP_H22 fieldmap. POP_H22 is a field in the KIHON_Chugoku2010 feature class.
    # The output will have the PHCCover Polygon with the attributes of the KIHON_Chugoku2010. Setting the
    # field's merge rule to SUM will aggregate the values for all of the KIHON_Chugoku2010 for
    # each break time into an SUM value. The field is also renamed to be more appropriate
    # for the output.
    POP2010FieldIndex = fieldmappings.findFieldMapIndex("POP_H22")
    fieldmap = fieldmappings.getFieldMap(POP2010FieldIndex)
    
    # Get the output field's properties as a field object
    field = fieldmap.outputField
    
    # Rename the field and pass the updated field object back into the field map
    field.name = "SUM_POP"
    field.aliasName = "SUM_POP"
    fieldmap.outputField = field
    
    # Set the merge rule to sum and then replace the old fieldmap in the mappings object
    # with the updated one
    fieldmap.mergeRule = "sum"
    fieldmappings.replaceFieldMap(POP2010FieldIndex, fieldmap)
    
    # Delete fields that are no longer needed, such as city YEAR and PREF
    # as only the first value will be used by default
    x = fieldmappings.findFieldMapIndex("YEAR")
    fieldmappings.removeFieldMap(x)
    y = fieldmappings.findFieldMapIndex("PREF")
    fieldmappings.removeFieldMap(y)
    z = fieldmappings.findFieldMapIndex("CITY")
    fieldmappings.removeFieldMap(z)
    a = fieldmappings.findFieldMapIndex("K_CODE")
    fieldmappings.removeFieldMap(a)
    b = fieldmappings.findFieldMapIndex("C_CODE")
    fieldmappings.removeFieldMap(b)
    c = fieldmappings.findFieldMapIndex("DID")
    fieldmappings.removeFieldMap(c)
    d = fieldmappings.findFieldMapIndex("KEY_CODE")
    fieldmappings.removeFieldMap(d)
    e = fieldmappings.findFieldMapIndex("PREF_NAME")
    fieldmappings.removeFieldMap(e)
    f = fieldmappings.findFieldMapIndex("CITY_NAME")
    fieldmappings.removeFieldMap(f)
    g = fieldmappings.findFieldMapIndex("OAZA_NAME")
    fieldmappings.removeFieldMap(g)
    h = fieldmappings.findFieldMapIndex("AZA_NAME")
    fieldmappings.removeFieldMap(h)
    i = fieldmappings.findFieldMapIndex("MALE_H22")
    fieldmappings.removeFieldMap(i)
    j = fieldmappings.findFieldMapIndex("FEMALE_H22")
    fieldmappings.removeFieldMap(j)
    k = fieldmappings.findFieldMapIndex("HOUSEHOLDS_H22")
    fieldmappings.removeFieldMap(k)
    l = fieldmappings.findFieldMapIndex("Name")
    fieldmappings.removeFieldMap(l)
    
    
    #Run the Spatial Join tool, using the defaults for the join operation and join type
    arcpy.SpatialJoin_analysis(targetFeatures, joinFeatures, outfc, "JOIN_ONE_TO_ONE", "KEEP_ALL", fieldmappings,"COMPLETELY_CONTAINS")

    print("Script completed successfully")
    
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))
    

Script completed successfully


### Hos2

In [2]:
### Import system modules
import arcpy
import os

try:
    
    # Set local variables
    inWorkspace1 = r"C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/NetworkServiceRural.gdb"
    inWorkspace2 = r"C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/HealthPlaceNew.gdb"
    outWorkspace = r"C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/E2FCAStep1Rural.gdb"
      
    # Want to join PHCCover polygon to KIHON_Chugoku2010 and calculate
    # the total number of population that service area (10, 20, 30 min) can cover
    targetFeatures = os.path.join(inWorkspace1, "SAPolygons2")
    joinFeatures = os.path.join(inWorkspace2, "KIHON_Chugoku2010")
    
    # Output will be the target features: PHCCover Polygon with a sum of population field 
    outfc = os.path.join(outWorkspace, "Hos2CoverPop")
    
    # Create a new fieldmappings and add the two input feature classes.
    fieldmappings = arcpy.FieldMappings()
    fieldmappings.addTable(targetFeatures)
    fieldmappings.addTable(joinFeatures)
    
    # First get the POP_H22 fieldmap. POP_H22 is a field in the KIHON_Chugoku2010 feature class.
    # The output will have the PHCCover Polygon with the attributes of the KIHON_Chugoku2010. Setting the
    # field's merge rule to SUM will aggregate the values for all of the KIHON_Chugoku2010 for
    # each break time into an SUM value. The field is also renamed to be more appropriate
    # for the output.
    POP2010FieldIndex = fieldmappings.findFieldMapIndex("POP_H22")
    fieldmap = fieldmappings.getFieldMap(POP2010FieldIndex)
    
    # Get the output field's properties as a field object
    field = fieldmap.outputField
    
    # Rename the field and pass the updated field object back into the field map
    field.name = "SUM_POP"
    field.aliasName = "SUM_POP"
    fieldmap.outputField = field
    
    # Set the merge rule to sum and then replace the old fieldmap in the mappings object
    # with the updated one
    fieldmap.mergeRule = "sum"
    fieldmappings.replaceFieldMap(POP2010FieldIndex, fieldmap)
    
    # Delete fields that are no longer applicable, such as city CITY_NAME and CITY_FIPS
    # as only the first value will be used by default
    x = fieldmappings.findFieldMapIndex("YEAR")
    fieldmappings.removeFieldMap(x)
    y = fieldmappings.findFieldMapIndex("PREF")
    fieldmappings.removeFieldMap(y)
    z = fieldmappings.findFieldMapIndex("CITY")
    fieldmappings.removeFieldMap(z)
    a = fieldmappings.findFieldMapIndex("K_CODE")
    fieldmappings.removeFieldMap(a)
    b = fieldmappings.findFieldMapIndex("C_CODE")
    fieldmappings.removeFieldMap(b)
    c = fieldmappings.findFieldMapIndex("DID")
    fieldmappings.removeFieldMap(c)
    d = fieldmappings.findFieldMapIndex("KEY_CODE")
    fieldmappings.removeFieldMap(d)
    e = fieldmappings.findFieldMapIndex("PREF_NAME")
    fieldmappings.removeFieldMap(e)
    f = fieldmappings.findFieldMapIndex("CITY_NAME")
    fieldmappings.removeFieldMap(f)
    g = fieldmappings.findFieldMapIndex("OAZA_NAME")
    fieldmappings.removeFieldMap(g)
    h = fieldmappings.findFieldMapIndex("AZA_NAME")
    fieldmappings.removeFieldMap(h)
    i = fieldmappings.findFieldMapIndex("MALE_H22")
    fieldmappings.removeFieldMap(i)
    j = fieldmappings.findFieldMapIndex("FEMALE_H22")
    fieldmappings.removeFieldMap(j)
    k = fieldmappings.findFieldMapIndex("HOUSEHOLDS_H22")
    fieldmappings.removeFieldMap(k)
    l = fieldmappings.findFieldMapIndex("Name")
    fieldmappings.removeFieldMap(l)
    
    #Run the Spatial Join tool, using the defaults for the join operation and join type
    arcpy.SpatialJoin_analysis(targetFeatures, joinFeatures, outfc, "JOIN_ONE_TO_ONE", "KEEP_ALL", fieldmappings,"COMPLETELY_CONTAINS")

    print("Script completed successfully")
    
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


### Hos3

In [3]:
### Import system modules
import arcpy
import os

try:
    
    # Set local variables
    inWorkspace1 = r"C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/NetworkServiceRural.gdb"
    inWorkspace2 = r"C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/HealthPlaceNew.gdb"
    outWorkspace = r"C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/E2FCAStep1Rural.gdb"
    
    # Want to join PHCCover polygon to KIHON_Chugoku2010 and calculate
    # the total number of population that service area (10, 20, 30 min) can cover
    targetFeatures = os.path.join(inWorkspace1, "SAPolygons3")
    joinFeatures = os.path.join(inWorkspace2, "KIHON_Chugoku2010")
    
    # Output will be the target features: PHCCover Polygon with a sum of population field 
    outfc = os.path.join(outWorkspace, "Hos3CoverPop")
    
    # Create a new fieldmappings and add the two input feature classes.
    fieldmappings = arcpy.FieldMappings()
    fieldmappings.addTable(targetFeatures)
    fieldmappings.addTable(joinFeatures)
    
    # First get the POP_H22 fieldmap. POP_H22 is a field in the KIHON_Chugoku2010 feature class.
    # The output will have the PHCCover Polygon with the attributes of the KIHON_Chugoku2010. Setting the
    # field's merge rule to SUM will aggregate the values for all of the KIHON_Chugoku2010 for
    # each break time into an SUM value. The field is also renamed to be more appropriate
    # for the output.
    POP2010FieldIndex = fieldmappings.findFieldMapIndex("POP_H22")
    fieldmap = fieldmappings.getFieldMap(POP2010FieldIndex)
    
    # Get the output field's properties as a field object
    field = fieldmap.outputField
    
    # Rename the field and pass the updated field object back into the field map
    field.name = "SUM_POP"
    field.aliasName = "SUM_POP"
    fieldmap.outputField = field
    
    # Set the merge rule to sum and then replace the old fieldmap in the mappings object
    # with the updated one
    fieldmap.mergeRule = "sum"
    fieldmappings.replaceFieldMap(POP2010FieldIndex, fieldmap)
    
    # Delete fields that are no longer applicable, such as city CITY_NAME and CITY_FIPS
    # as only the first value will be used by default
    x = fieldmappings.findFieldMapIndex("YEAR")
    fieldmappings.removeFieldMap(x)
    y = fieldmappings.findFieldMapIndex("PREF")
    fieldmappings.removeFieldMap(y)
    z = fieldmappings.findFieldMapIndex("CITY")
    fieldmappings.removeFieldMap(z)
    a = fieldmappings.findFieldMapIndex("K_CODE")
    fieldmappings.removeFieldMap(a)
    b = fieldmappings.findFieldMapIndex("C_CODE")
    fieldmappings.removeFieldMap(b)
    c = fieldmappings.findFieldMapIndex("DID")
    fieldmappings.removeFieldMap(c)
    d = fieldmappings.findFieldMapIndex("KEY_CODE")
    fieldmappings.removeFieldMap(d)
    e = fieldmappings.findFieldMapIndex("PREF_NAME")
    fieldmappings.removeFieldMap(e)
    f = fieldmappings.findFieldMapIndex("CITY_NAME")
    fieldmappings.removeFieldMap(f)
    g = fieldmappings.findFieldMapIndex("OAZA_NAME")
    fieldmappings.removeFieldMap(g)
    h = fieldmappings.findFieldMapIndex("AZA_NAME")
    fieldmappings.removeFieldMap(h)
    i = fieldmappings.findFieldMapIndex("MALE_H22")
    fieldmappings.removeFieldMap(i)
    j = fieldmappings.findFieldMapIndex("FEMALE_H22")
    fieldmappings.removeFieldMap(j)
    k = fieldmappings.findFieldMapIndex("HOUSEHOLDS_H22")
    fieldmappings.removeFieldMap(k)
    l = fieldmappings.findFieldMapIndex("Name")
    fieldmappings.removeFieldMap(l)
    
    #Run the Spatial Join tool, using the defaults for the join operation and join type
    arcpy.SpatialJoin_analysis(targetFeatures, joinFeatures, outfc, "JOIN_ONE_TO_ONE", "KEEP_ALL", fieldmappings,"COMPLETELY_CONTAINS")

    print("Script completed successfully")
    
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


# Weighting the Step 1 population with slow and quick distance decay

### PHC

In [4]:
import arcpy
from arcpy import env
try:
    # Set environment settings
    env.workspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\E2FCAStep1Rural.gdb"
    
    # Set local variables
    inTable = "PHCCoverPop"
    fieldName1 = "PropToPopSlow"
    fieldName2 = "PropToPopQuick"
    expression1 = "w(!ToBreak!,!SUM_POP!)"
    codeblock1 = """def w (f1,f2):
        if f1 <= 10:
            return f2 * 1
        elif f1 > 10 and f1 <= 20:
            return f2 * 0.80
        elif f1 > 20 and f1 <=30:
            return f2 * 0.55
        elif f1 > 30 and f1 <=60:
            return f2 * 0.15"""
    codeblock2 = """def w(f1,f2):
        if f1 <= 10:
            return f2 * 1
        elif f1 > 10 and f1 <= 20:
            return f2 * 0.60
        elif f1 > 20 and f1 <=30:
            return f2 * 0.25
        elif f1 > 30 and f1 <=60:
            return f2 * 0.05"""
    
    # Execute AddField
    arcpy.AddField_management(inTable, fieldName1, "DOUBLE")
    arcpy.AddField_management(inTable, fieldName2, "DOUBLE")
    # Execute CalculateField - weighting
    arcpy.CalculateField_management(inTable, fieldName1, expression1, "PYTHON_9.3", codeblock1)
    arcpy.CalculateField_management(inTable, fieldName2, expression1, "PYTHON_9.3", codeblock2)

    print("Script completed successfully")
    
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


### Hos2

In [11]:
import arcpy
from arcpy import env
try:
    # Set environment settings
    env.workspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\E2FCAStep1Rural.gdb"
    
    # Set local variables
    inTable = "Hos2CoverPop"
    fieldName1 = "PropToPopSlow"
    fieldName2 = "PropToPopQuick"
    expression1 = "w(!ToBreak!,!SUM_POP!)"
    codeblock1 = """def w (f1,f2):
        if f1 <= 10:
            return f2 * 1
        elif f1 > 10 and f1 <= 20:
            return f2 * 0.80
        elif f1 > 20 and f1 <=30:
            return f2 * 0.55
        elif f1 > 30 and f1 <=60:
            return f2 * 0.15"""
    codeblock2 = """def w(f1,f2):
        if f1 <= 10:
            return f2 * 1
        elif f1 > 10 and f1 <= 20:
            return f2 * 0.60
        elif f1 > 20 and f1 <=30:
            return f2 * 0.25
        elif f1 > 30 and f1 <=60:
            return f2 * 0.05"""
    
    # Execute AddField
    arcpy.AddField_management(inTable, fieldName1, "DOUBLE")
    arcpy.AddField_management(inTable, fieldName2, "DOUBLE")
    # Execute CalculateField - weighting
    arcpy.CalculateField_management(inTable, fieldName1, expression1, "PYTHON_9.3", codeblock1)
    arcpy.CalculateField_management(inTable, fieldName2, expression1, "PYTHON_9.3", codeblock2)

    print("Script completed successfully")
    
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


### Hos3

In [12]:
import arcpy
from arcpy import env
try:
    # Set environment settings
    env.workspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\E2FCAStep1Rural.gdb"
    
    # Set local variables
    inTable = "Hos3CoverPop"
    fieldName1 = "PropToPopSlow"
    fieldName2 = "PropToPopQuick"
    expression1 = "w(!ToBreak!,!SUM_POP!)"
    codeblock1 = """def w (f1,f2):
        if f1 <= 10:
            return f2 * 1
        elif f1 > 10 and f1 <= 20:
            return f2 * 0.80
        elif f1 > 20 and f1 <=30:
            return f2 * 0.55
        elif f1 > 30 and f1 <=60:
            return f2 * 0.15"""
    codeblock2 = """def w(f1,f2):
        if f1 <= 10:
            return f2 * 1
        elif f1 > 10 and f1 <= 20:
            return f2 * 0.60
        elif f1 > 20 and f1 <=30:
            return f2 * 0.25
        elif f1 > 30 and f1 <=60:
            return f2 * 0.05"""
    
    # Execute AddField
    arcpy.AddField_management(inTable, fieldName1, "DOUBLE")
    arcpy.AddField_management(inTable, fieldName2, "DOUBLE")
    # Execute CalculateField - weighting
    arcpy.CalculateField_management(inTable, fieldName1, expression1, "PYTHON_9.3", codeblock1)
    arcpy.CalculateField_management(inTable, fieldName2, expression1, "PYTHON_9.3", codeblock2)

    print("Script completed successfully")
    
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


# Dissolve all 4 breaks polygons into one by Faciliy ID

## First block executes dissolve by Facility ID and sum up weighted population in each breaks
## Second block executes a calculation to get supply - demand ratio: 1 / cover pop

### PHC

In [13]:
import arcpy
from arcpy import env
import os
try:
    # Set environment settings
    env.workspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\E2FCAStep1Rural.gdb"
    # Set local variables
    inWorkspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\E2FCAStep1Rural.gdb"
    outWorkspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\E2FCAStep1Rural.gdb"
    inFeatures = os.path.join(inWorkspace,"PHCCoverPop")
    outFeatures = os.path.join(outWorkspace,"PHCCoverDiss")
    dissolveFields = ["FacilityID"]
    statFields = [["PropToPopSlow","sum"],["PropToPopQuick","sum"]]
    
    # Execute Dissolve using FaclityID as Dissolve Fields, summing the weighted PropToPop(Slow & Quick) of 3 time breaks into 1
    arcpy.Dissolve_management(inFeatures, outFeatures, dissolveFields, statFields)
    
    print("Script completed successfully")
    
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


In [16]:
import arcpy
from arcpy import env
try:
    # Set environment settings
    env.workspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\E2FCAStep1Rural.gdb"
    
    # Set local variables
    inTable = "PHCCoverDiss"
    fieldName1 = "SUM_PropToPopSlow"
    fieldName2 = "SUM_PropToPopQuick"
    expression1 = "1/!SUM_PropToPopSlow!"
    expression2 = "1/!SUM_PropToPopQuick!"    
    # Execute CalculateField - getting PropToPop deviding 1 by weighted population
    arcpy.CalculateField_management(inTable, fieldName1, expression1, "PYTHON_9.3")
    arcpy.CalculateField_management(inTable, fieldName2, expression2, "PYTHON_9.3")

    print("Script completed successfully")
    
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


### Hos2

In [14]:
import arcpy
from arcpy import env
import os
try:
    # Set environment settings
    env.workspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\E2FCAStep1Rural.gdb"
    # Set local variables
    inWorkspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\E2FCAStep1Rural.gdb"
    outWorkspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\E2FCAStep1Rural.gdb"
    inFeatures = os.path.join(inWorkspace,"Hos2CoverPop")
    outFeatures = os.path.join(outWorkspace,"Hos2CoverDiss")
    dissolveFields = ["FacilityID"]
    statFields = [["PropToPopSlow","sum"],["PropToPopQuick","sum"]]
    
    # Execute Dissolve using FaclityID as Dissolve Fields, summing the weighted PropToPop(Slow & Quick) of 3 time breaks into 1
    arcpy.Dissolve_management(inFeatures, outFeatures, dissolveFields, statFields)

    print("Script completed successfully")
    
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


In [17]:
import arcpy
from arcpy import env
try:
    # Set environment settings
    env.workspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\E2FCAStep1Rural.gdb"
    
    # Set local variables
    inTable = "Hos2CoverDiss"
    fieldName1 = "SUM_PropToPopSlow"
    fieldName2 = "SUM_PropToPopQuick"
    expression1 = "1/!SUM_PropToPopSlow!"
    expression2 = "1/!SUM_PropToPopQuick!"    
    # Execute CalculateField - getting PropToPop deviding 1 by weighted population
    arcpy.CalculateField_management(inTable, fieldName1, expression1, "PYTHON_9.3")
    arcpy.CalculateField_management(inTable, fieldName2, expression2, "PYTHON_9.3")

    print("Script completed successfully")
    
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


### Hos3

In [15]:
import arcpy
from arcpy import env
import os
try:
    # Set environment settings
    env.workspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\E2FCAStep1Rural.gdb"
    # Set local variables   
    inWorkspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\E2FCAStep1Rural.gdb"
    outWorkspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\E2FCAStep1Rural.gdb"
    inFeatures = os.path.join(inWorkspace,"Hos3CoverPop")
    outFeatures = os.path.join(outWorkspace,"Hos3CoverDiss")
    dissolveFields = ["FacilityID"]
    statFields = [["PropToPopSlow","sum"],["PropToPopQuick","sum"]]
    
    # Execute Dissolve using FaclityID as Dissolve Fields, summing the weighted PropToPop(Slow & Quick) of 3 time breaks into 1
    arcpy.Dissolve_management(inFeatures, outFeatures, dissolveFields, statFields)

    print("Script completed successfully")
    
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


In [18]:
import arcpy
from arcpy import env
try:
    # Set environment settings
    env.workspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\E2FCAStep1Rural.gdb"
    
    # Set local variables
    inTable = "Hos3CoverDiss"
    fieldName1 = "SUM_PropToPopSlow"
    fieldName2 = "SUM_PropToPopQuick"
    expression1 = "1/!SUM_PropToPopSlow!"
    expression2 = "1/!SUM_PropToPopQuick!"    
    # Execute CalculateField - getting PropToPop deviding 1 by weighted population
    arcpy.CalculateField_management(inTable, fieldName1, expression1, "PYTHON_9.3")
    arcpy.CalculateField_management(inTable, fieldName2, expression2, "PYTHON_9.3")

    print("Script completed successfully")
    
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


# Join fields (PropToPopSlow and PropToPopQuick) back to the healthcare point feature

### PHC

In [19]:
import arcpy
from arcpy import env
import os
try:
    # Set environment settings
    env.workspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\E2FCAStep1Rural.gdb"
    
    # Set the local parameters
    inWorkspace1 = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\HealthPlaceNew.gdb"
    inWorkspace2 = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\E2FCAStep1Rural.gdb"
    inFeatures = os.path.join(inWorkspace1,"PHCNaika_Chugoku")
    joinField1 = "OBJECTID"
    joinTable = os.path.join(inWorkspace2,"PHCCoverDiss")
    joinField2 = "FacilityID"
    fieldList = ["SUM_PropToPopSlow", "SUM_PropToPopQuick"] 
    outFeatures = os.path.join(inWorkspace2,"PHCNaika_ChugokuPTP")
    dropFields = ["P04_001", "P04_002", "P04_003", "P04_004", "P04_005", "P04_006", "P04_007", "Naika"]
    
    
    # Join two feature classes by the OBJECTID and FacilityID fields and only carry 
    # over the PropToPopSlow and PropToPopQuick fields
    arcpy.JoinField_management(inFeatures, joinField1, joinTable, joinField2, fieldList)
    # Create a new feature in a current workspace = E2FCAStep1Rural.gdb
    arcpy.CopyFeatures_management(inFeatures, outFeatures)
    # Delete unnecessary fields
    arcpy.DeleteField_management(outFeatures, dropFields)
    
    print("Script completed successfully")

except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


### Hos2

In [20]:
import arcpy
from arcpy import env
import os
try:
    # Set environment settings
    env.workspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\E2FCAStep1Rural.gdb"
    
    # Set the local parameters
    inWorkspace1 = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\HealthPlaceNew.gdb"
    inWorkspace2 = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\E2FCAStep1Rural.gdb"
    inFeatures = os.path.join(inWorkspace1,"Hos2Naika_Chugoku")
    joinField1 = "OBJECTID"
    joinTable = os.path.join(inWorkspace2,"Hos2CoverDiss")
    joinField2 = "FacilityID"
    fieldList = ["SUM_PropToPopSlow", "SUM_PropToPopQuick"] 
    outFeatures = os.path.join(inWorkspace2,"Hos2Naika_ChugokuPTP")
    dropFields = ["P04_001", "P04_002", "P04_003", "P04_004", "P04_005", "P04_006", "P04_007", "Naika"]
    
    
    # Join two feature classes by the OBJECTID and FacilityID fields and only carry 
    # over the PropToPopSlow and PropToPopQuick fields
    arcpy.JoinField_management(inFeatures, joinField1, joinTable, joinField2, fieldList)
    # Create a new feature in a current workspace = E2FCAStep1Rural.gdb
    arcpy.CopyFeatures_management(inFeatures, outFeatures)
    # Delete unnecessary fields
    arcpy.DeleteField_management(outFeatures, dropFields)
    
    print("Script completed successfully")

except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


### Hos3

In [21]:
import arcpy
from arcpy import env
import os
try:
    # Set environment settings
    env.workspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\E2FCAStep1Rural.gdb"
    
    # Set the local parameters
    inWorkspace1 = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\HealthPlaceNew.gdb"
    inWorkspace2 = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\E2FCAStep1Rural.gdb"
    inFeatures = os.path.join(inWorkspace1,"Hos3Naika_Chugoku")
    joinField1 = "OBJECTID"
    joinTable = os.path.join(inWorkspace2,"Hos3CoverDiss")
    joinField2 = "FacilityID"
    fieldList = ["SUM_PropToPopSlow", "SUM_PropToPopQuick"] 
    outFeatures = os.path.join(inWorkspace2,"Hos3Naika_ChugokuPTP")
    dropFields = ["P04_001", "P04_002", "P04_003", "P04_004", "P04_005", "P04_006", "P04_007", "Naika"]
    
    
    # Join two feature classes by the OBJECTID and FacilityID fields and only carry 
    # over the PropToPopSlow and PropToPopQuick fields
    arcpy.JoinField_management(inFeatures, joinField1, joinTable, joinField2, fieldList)
    # Create a new feature in a current workspace = E2FCAStep1Rural.gdb
    arcpy.CopyFeatures_management(inFeatures, outFeatures)
    # Delete unnecessary fields
    arcpy.DeleteField_management(outFeatures, dropFields)
    
    print("Script completed successfully")

except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


# Create network service layer for Centroid point of cho-cho aza

### Centroid gravity point of residential area

In [ ]:
import arcpy
from arcpy import env
import os
try:
    #Set environment settings
    output_dir = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew"
    #The NA layer's data will be saved to the workspace specified here
    env.workspace = os.path.join(output_dir, "NetworkServiceRural.gdb")
    env.overwriteOutput = True
    
    #Set local variables
    input_gdb1 = r"C:/Users/kenta/Documents/HDDFiles/GISData_Import/Esri道路2014中国/道路網_鳥取県_島根県_岡山県_広島県_山口県版/ADF2014.gdb"
    network = os.path.join(input_gdb1, "ADF","ADF_ND")
    layer_name = "CentroidResAreaCover"
    travel_mode = "Driving Time"
    input_gdb2 = "C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/HealthPlaceNew.gdb"
    facilities = os.path.join(input_gdb2,"Census2015Res_Centroid")
    output_layer_file = os.path.join(output_dir, layer_name + ".lyrx")
  
    
    #Create a new service area layer. We wish to generate the service area
    #polygons as rings, so that we can easily visualize the coverage for any
    #given location. We also want overlapping polygons so we can determine the
    #number of fire stations that cover a given location. We will specify these
    #options while creating the new service area layer.
    result_object = arcpy.na.MakeServiceAreaAnalysisLayer(network, layer_name,
                                    travel_mode, "FROM_FACILITIES", [10, 20, 30, 60],
                                    time_zone="LOCAL_TIME_AT_LOCATIONS",
                                    polygon_detail="STANDARD",
                                    geometry_at_overlaps="OVERLAP",
                                    geometry_at_cutoffs="RINGS")

    #Get the layer object from the result object. The service layer can now be
    #referenced using the layer object.
    layer_object = result_object.getOutput(0)

    #Get the names of all the sublayers within the service area layer.
    sublayer_names = arcpy.na.GetNAClassNames(layer_object)
    #Stores the layer names that we will use later
    facilities_layer_name = sublayer_names["Facilities"]

    #Load the fire stations as facilities using default field mappings and
    #default search tolerance
    arcpy.na.AddLocations(layer_object, facilities_layer_name, facilities, "", "")

    #Solve the service area layer
    arcpy.na.Solve(layer_object)

    #Save the solved service area layer as a layer file on disk
    layer_object.saveACopy(output_layer_file)

    print("Script completed successfully")

except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

### Simple centroid point

In [22]:
import arcpy
from arcpy import env
import os
try:
    #Set environment settings
    output_dir = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew"
    #The NA layer's data will be saved to the workspace specified here
    env.workspace = os.path.join(output_dir, "NetworkServiceRural.gdb")
    env.overwriteOutput = True
    
    #Set local variables
    input_gdb1 = r"C:/Users/kenta/Documents/HDDFiles/GISData_Import/Esri道路2014中国/道路網_鳥取県_島根県_岡山県_広島県_山口県版/ADF2014.gdb"
    network = os.path.join(input_gdb1, "ADF","ADF_ND")
    layer_name = "CentroidSimpleCover"
    travel_mode = "Driving Time"
    input_gdb2 = "C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/HealthPlaceNew.gdb"
    facilities = os.path.join(input_gdb2,"Census2015Centroid")
    output_layer_file = os.path.join(output_dir, layer_name + ".lyrx")
  
    
    #Create a new service area layer. We wish to generate the service area
    #polygons as rings, so that we can easily visualize the coverage for any
    #given location. We also want overlapping polygons so we can determine the
    #number of fire stations that cover a given location. We will specify these
    #options while creating the new service area layer.
    result_object = arcpy.na.MakeServiceAreaAnalysisLayer(network, layer_name,
                                    travel_mode, "FROM_FACILITIES", [10, 20, 30, 60],
                                    time_zone="LOCAL_TIME_AT_LOCATIONS",
                                    polygon_detail="STANDARD",
                                    geometry_at_overlaps="OVERLAP",
                                    geometry_at_cutoffs="RINGS")

    #Get the layer object from the result object. The service layer can now be
    #referenced using the layer object.
    layer_object = result_object.getOutput(0)

    #Get the names of all the sublayers within the service area layer.
    sublayer_names = arcpy.na.GetNAClassNames(layer_object)
    #Stores the layer names that we will use later
    facilities_layer_name = sublayer_names["Facilities"]

    #Load the fire stations as facilities using default field mappings and
    #default search tolerance
    arcpy.na.AddLocations(layer_object, facilities_layer_name, facilities, "", "")

    #Solve the service area layer
    arcpy.na.Solve(layer_object)

    #Save the solved service area layer as a layer file on disk
    layer_object.saveACopy(output_layer_file)

    print("Script completed successfully")

except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


# Spatial join with healthcare facilities point data

### PHC

In [23]:
### Import system modules
import arcpy
import os

try:
    
    # Set local variables
    inWorkspace1 = r"C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/NetworkServiceRural.gdb"
    inWorkspace2 = r"C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/E2FCAStep1Rural.gdb"
    outWorkspace = r"C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/E2FCAStep2Rural.gdb"
    
    # Want to join Centroid catchment area polygon to Step1 PHC data and calculate
    # the total score that catchment area (10, 20, 30 min) can cover
    targetFeatures = os.path.join(inWorkspace1, "SAPolygons4")
    joinFeatures = os.path.join(inWorkspace2, "PHCNaika_ChugokuPTP")
    
    # Output will be the target features: PHCCover Polygon with a sum of population field 
    outfc = os.path.join(outWorkspace, "CentroidCatchPHC")
    
    # Create a new fieldmappings and add the two input feature classes.
    fieldmappings = arcpy.FieldMappings()
    fieldmappings.addTable(targetFeatures)
    fieldmappings.addTable(joinFeatures)
    
    # First get the POP_H22 fieldmap. POP_H22 is a field in the KIHON_Chugoku2010 feature class.
    # The output will have the PHCCover Polygon with the attributes of the KIHON_Chugoku2010. Setting the
    # field's merge rule to SUM will aggregate the values for all of the KIHON_Chugoku2010 for
    # each break time into an SUM value. The field is also renamed to be more appropriate
    # for the output.
    PropToPopSlowFieldIndex = fieldmappings.findFieldMapIndex("SUM_PropToPopSlow")
    PropToPopQuickFieldIndex = fieldmappings.findFieldMapIndex("SUM_PropToPopQuick")
    fieldmapSlow = fieldmappings.getFieldMap(PropToPopSlowFieldIndex)
    fieldmapQuick = fieldmappings.getFieldMap(PropToPopQuickFieldIndex)
    # Get the output field's properties as a field object
    fieldS = fieldmapSlow.outputField
    fieldQ = fieldmapQuick.outputField
    
    # Rename the field and pass the updated field object back into the field map
    fieldS.name = "E2FCAPHCSlow"
    fieldS.aliasName = "E2FCAPHCSlow"
    fieldQ.name = "E2FCAPHCQuick"
    fieldQ.aliasName = "E2FCAPHCQuick"
    fieldmapSlow.outputField = fieldS
    fieldmapQuick.outputField = fieldQ
    
    # Set the merge rule to sum and then replace the old fieldmap in the mappings object
    # with the updated one
    fieldmapSlow.mergeRule = "sum"
    fieldmapQuick.mergeRule = "sum"
    fieldmappings.replaceFieldMap(PropToPopSlowFieldIndex, fieldmapSlow)
    fieldmappings.replaceFieldMap(PropToPopQuickFieldIndex, fieldmapQuick)
    
    # Delete fields that are no longer applicable, such as city CITY_NAME and CITY_FIPS
    # as only the first value will be used by default
    x = fieldmappings.findFieldMapIndex("Name")
    fieldmappings.removeFieldMap(x)
    
    
    #Run the Spatial Join tool, using the defaults for the join operation and join type
    arcpy.SpatialJoin_analysis(targetFeatures, joinFeatures, outfc, "JOIN_ONE_TO_ONE", "KEEP_ALL", fieldmappings,"COMPLETELY_CONTAINS")

    print("Script completed successfully")
    
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


### Hos2

In [24]:
### Import system modules
import arcpy
import os

try:
    
    # Set local variables
    inWorkspace1 = r"C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/NetworkServiceRural.gdb"
    inWorkspace2 = r"C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/E2FCAStep1Rural.gdb"
    outWorkspace = r"C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/E2FCAStep2Rural.gdb"
    
    # Want to join Centroid catchment area polygon to Step1 PHC data and calculate
    # the total score that catchment area (10, 20, 30 min) can cover
    targetFeatures = os.path.join(inWorkspace1, "SAPolygons4")
    joinFeatures = os.path.join(inWorkspace2, "Hos2Naika_ChugokuPTP")
    
    # Output will be the target features: PHCCover Polygon with a sum of population field 
    outfc = os.path.join(outWorkspace, "CentroidCatchHos2")
    
    # Create a new fieldmappings and add the two input feature classes.
    fieldmappings = arcpy.FieldMappings()
    fieldmappings.addTable(targetFeatures)
    fieldmappings.addTable(joinFeatures)
    
    # First get the POP_H22 fieldmap. POP_H22 is a field in the KIHON_Chugoku2010 feature class.
    # The output will have the PHCCover Polygon with the attributes of the KIHON_Chugoku2010. Setting the
    # field's merge rule to SUM will aggregate the values for all of the KIHON_Chugoku2010 for
    # each break time into an SUM value. The field is also renamed to be more appropriate
    # for the output.
    PropToPopSlowFieldIndex = fieldmappings.findFieldMapIndex("SUM_PropToPopSlow")
    PropToPopQuickFieldIndex = fieldmappings.findFieldMapIndex("SUM_PropToPopQuick")
    fieldmapSlow = fieldmappings.getFieldMap(PropToPopSlowFieldIndex)
    fieldmapQuick = fieldmappings.getFieldMap(PropToPopQuickFieldIndex)
    # Get the output field's properties as a field object
    fieldS = fieldmapSlow.outputField
    fieldQ = fieldmapQuick.outputField
    
    # Rename the field and pass the updated field object back into the field map
    fieldS.name = "E2FCAHos2Slow"
    fieldS.aliasName = "E2FCAHos2Slow"
    fieldQ.name = "E2FCAHos2Quick"
    fieldQ.aliasName = "E2FCAHos2Quick"
    fieldmapSlow.outputField = fieldS
    fieldmapQuick.outputField = fieldQ
    
    # Set the merge rule to sum and then replace the old fieldmap in the mappings object
    # with the updated one
    fieldmapSlow.mergeRule = "sum"
    fieldmapQuick.mergeRule = "sum"
    fieldmappings.replaceFieldMap(PropToPopSlowFieldIndex, fieldmapSlow)
    fieldmappings.replaceFieldMap(PropToPopQuickFieldIndex, fieldmapQuick)
    
    # Delete fields that are no longer applicable, such as city CITY_NAME and CITY_FIPS
    # as only the first value will be used by default
    x = fieldmappings.findFieldMapIndex("Name")
    fieldmappings.removeFieldMap(x)
    
    
    #Run the Spatial Join tool, using the defaults for the join operation and join type
    arcpy.SpatialJoin_analysis(targetFeatures, joinFeatures, outfc, "JOIN_ONE_TO_ONE", "KEEP_ALL", fieldmappings,"COMPLETELY_CONTAINS")

    print("Script completed successfully")
        
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


In [ ]:
# Hos3

In [25]:
### Import system modules
import arcpy
import os

try:
    
    # Set local variables
    inWorkspace1 = r"C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/NetworkServiceRural.gdb"
    inWorkspace2 = r"C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/E2FCAStep1Rural.gdb"
    outWorkspace = r"C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/E2FCAStep2Rural.gdb"
    
    # Want to join Centroid catchment area polygon to Step1 PHC data and calculate
    # the total score that catchment area (10, 20, 30 min) can cover
    targetFeatures = os.path.join(inWorkspace1, "SAPolygons4")
    joinFeatures = os.path.join(inWorkspace2, "Hos3Naika_ChugokuPTP")
    
    # Output will be the target features: PHCCover Polygon with a sum of population field 
    outfc = os.path.join(outWorkspace, "CentroidCatchHos3")
    
    # Create a new fieldmappings and add the two input feature classes.
    fieldmappings = arcpy.FieldMappings()
    fieldmappings.addTable(targetFeatures)
    fieldmappings.addTable(joinFeatures)
    
    # First get the POP_H22 fieldmap. POP_H22 is a field in the KIHON_Chugoku2010 feature class.
    # The output will have the PHCCover Polygon with the attributes of the KIHON_Chugoku2010. Setting the
    # field's merge rule to SUM will aggregate the values for all of the KIHON_Chugoku2010 for
    # each break time into an SUM value. The field is also renamed to be more appropriate
    # for the output.
    PropToPopSlowFieldIndex = fieldmappings.findFieldMapIndex("SUM_PropToPopSlow")
    PropToPopQuickFieldIndex = fieldmappings.findFieldMapIndex("SUM_PropToPopQuick")
    fieldmapSlow = fieldmappings.getFieldMap(PropToPopSlowFieldIndex)
    fieldmapQuick = fieldmappings.getFieldMap(PropToPopQuickFieldIndex)
    # Get the output field's properties as a field object
    fieldS = fieldmapSlow.outputField
    fieldQ = fieldmapQuick.outputField
    
    # Rename the field and pass the updated field object back into the field map
    fieldS.name = "E2FCAHos3Slow"
    fieldS.aliasName = "E2FCAHos3Slow"
    fieldQ.name = "E2FCAHos3Quick"
    fieldQ.aliasName = "E2FCAHos3Quick"
    fieldmapSlow.outputField = fieldS
    fieldmapQuick.outputField = fieldQ
    
    # Set the merge rule to sum and then replace the old fieldmap in the mappings object
    # with the updated one
    fieldmapSlow.mergeRule = "sum"
    fieldmapQuick.mergeRule = "sum"
    fieldmappings.replaceFieldMap(PropToPopSlowFieldIndex, fieldmapSlow)
    fieldmappings.replaceFieldMap(PropToPopQuickFieldIndex, fieldmapQuick)
    
    # Delete fields that are no longer applicable, such as city CITY_NAME and CITY_FIPS
    # as only the first value will be used by default
    x = fieldmappings.findFieldMapIndex("Name")
    fieldmappings.removeFieldMap(x)
    
    
    #Run the Spatial Join tool, using the defaults for the join operation and join type
    arcpy.SpatialJoin_analysis(targetFeatures, joinFeatures, outfc, "JOIN_ONE_TO_ONE", "KEEP_ALL", fieldmappings,"COMPLETELY_CONTAINS")

    print("Script completed successfully")
    
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


# Weighting scores for all 4 breaks

### PHC

In [31]:
import arcpy
from arcpy import env
try:
    # Set environment settings
    env.workspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\E2FCAStep2Rural.gdb"
    
    # Set local variables
    inTable = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\E2FCAStep2Rural.gdb\CentroidCatchPHC"
    fieldName1 = "PHCE2FCASlow"
    fieldName2 = "PHCE2FCAQuick"
    expression1 = "w(!ToBreak!,!E2FCAPHCSlow!)"
    expression2 = "w(!ToBreak!,!E2FCAPHCQuick!)"
    codeblock1 = """def w (f1,f2):
        if f1 <= 10:
            return f2 * 1
        elif f1 > 10 and f1 <= 20:
            return f2 * 0.80
        elif f1 > 20 and f1 <=30:
            return f2 * 0.55
        elif f1 > 30 and f1 <=60:
            return f2 * 0.15"""
    codeblock2 = """def w(f1,f2):
        if f1 <= 10:
            return f2 * 1
        elif f1 > 10 and f1 <= 20:
            return f2 * 0.60
        elif f1 > 20 and f1 <=30:
            return f2 * 0.25
        elif f1 > 30 and f1 <=60:
            return f2 * 0.05"""   
    # Execute AddField
    arcpy.AddField_management(inTable, fieldName1, "DOUBLE")
    arcpy.AddField_management(inTable, fieldName2, "DOUBLE")
    # Execute CalculateField - weighting
    arcpy.CalculateField_management(inTable, fieldName1, expression1, "PYTHON_9.3", codeblock1)
    arcpy.CalculateField_management(inTable, fieldName2, expression2, "PYTHON_9.3", codeblock2)
    
    print("Script completed successfully")
    
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


### Hos2

In [32]:
import arcpy
from arcpy import env
try:
    # Set environment settings
    env.workspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\E2FCAStep2Rural.gdb"
    
    # Set local variables
    inTable = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\E2FCAStep2Rural.gdb\CentroidCatchHos2"
    fieldName1 = "Hos2E2FCASlow"
    fieldName2 = "Hos2E2FCAQuick"
    expression1 = "w(!ToBreak!,!E2FCAHos2Slow!)"
    expression2 = "w(!ToBreak!,!E2FCAHos2Quick!)"
    codeblock1 = """def w (f1,f2):
        if f1 <= 10:
            return f2 * 1
        elif f1 > 10 and f1 <= 20:
            return f2 * 0.80
        elif f1 > 20 and f1 <=30:
            return f2 * 0.55
        elif f1 > 30 and f1 <=60:
            return f2 * 0.15"""
    codeblock2 = """def w(f1,f2):
        if f1 <= 10:
            return f2 * 1
        elif f1 > 10 and f1 <= 20:
            return f2 * 0.60
        elif f1 > 20 and f1 <=30:
            return f2 * 0.25
        elif f1 > 30 and f1 <=60:
            return f2 * 0.05"""   
    # Execute AddField
    arcpy.AddField_management(inTable, fieldName1, "DOUBLE")
    arcpy.AddField_management(inTable, fieldName2, "DOUBLE")
    # Execute CalculateField - weighting
    arcpy.CalculateField_management(inTable, fieldName1, expression1, "PYTHON_9.3", codeblock1)
    arcpy.CalculateField_management(inTable, fieldName2, expression2, "PYTHON_9.3", codeblock2)

    print("Script completed successfully")

except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


### Hos3

In [33]:
import arcpy
from arcpy import env
try:
    # Set environment settings
    env.workspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\E2FCAStep2Rural.gdb"
    
    # Set local variables
    inTable = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\E2FCAStep2Rural.gdb\CentroidCatchHos3"
    fieldName1 = "Hos3E2FCASlow"
    fieldName2 = "Hos3E2FCAQuick"
    expression1 = "w(!ToBreak!,!E2FCAHos3Slow!)"
    expression2 = "w(!ToBreak!,!E2FCAHos3Quick!)"
    codeblock1 = """def w (f1,f2):
        if f1 <= 10:
            return f2 * 1
        elif f1 > 10 and f1 <= 20:
            return f2 * 0.80
        elif f1 > 20 and f1 <=30:
            return f2 * 0.55
        elif f1 > 30 and f1 <=60:
            return f2 * 0.15"""
    codeblock2 = """def w(f1,f2):
        if f1 <= 10:
            return f2 * 1
        elif f1 > 10 and f1 <= 20:
            return f2 * 0.60
        elif f1 > 20 and f1 <=30:
            return f2 * 0.25
        elif f1 > 30 and f1 <=60:
            return f2 * 0.05"""   
    # Execute AddField
    arcpy.AddField_management(inTable, fieldName1, "DOUBLE")
    arcpy.AddField_management(inTable, fieldName2, "DOUBLE")
    # Execute CalculateField - weighting
    arcpy.CalculateField_management(inTable, fieldName1, expression1, "PYTHON_9.3", codeblock1)
    arcpy.CalculateField_management(inTable, fieldName2, expression2, "PYTHON_9.3", codeblock2)

    print("Script completed successfully")

except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


# Dissolve by Facility ID and get E2FCA score 

### PHC

In [34]:
import arcpy
from arcpy import env
import os
try:
    # Set environment settings
    env.workspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\E2FCAStep2Rural.gdb"
    # Set local variables
    inWorkspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\E2FCAStep2Rural.gdb"
    outWorkspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\E2FCAStep2Rural.gdb"
    inFeatures = os.path.join(inWorkspace,"CentroidCatchPHC")
    outFeatures = os.path.join(outWorkspace,"CentroidCatchPHC_Diss")
    dissolveFields = ["FacilityID"]
    statFields = [["PHCE2FCASlow","sum"],["PHCE2FCAQuick","sum"]]
    
    # Execute Dissolve using FaclityID as Dissolve Fields, summing the weighted PropToPop(Slow & Quick) of 3 time breaks into 1
    arcpy.Dissolve_management(inFeatures, outFeatures, dissolveFields, statFields)

    print("Script completed successfully")
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


### Hos2

In [35]:
import arcpy
from arcpy import env
import os
try:
    # Set environment settings
    env.workspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\E2FCAStep2Rural.gdb"
    # Set local variables
    inWorkspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\E2FCAStep2Rural.gdb"
    outWorkspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\E2FCAStep2Rural.gdb"
    inFeatures = os.path.join(inWorkspace,"CentroidCatchHos2")
    outFeatures = os.path.join(outWorkspace,"CentroidCatchHos2_Diss")
    dissolveFields = ["FacilityID"]
    statFields = [["Hos2E2FCASlow","sum"],["Hos2E2FCAQuick","sum"]]
    
    # Execute Dissolve using FaclityID as Dissolve Fields, summing the weighted PropToPop(Slow & Quick) of 3 time breaks into 1
    arcpy.Dissolve_management(inFeatures, outFeatures, dissolveFields, statFields)

    print("Script completed successfully")
    
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


### Hos3

In [38]:
import arcpy
from arcpy import env
import os
try:
    # Set environment settings
    env.workspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\E2FCAStep2Rural.gdb"
    # Set local variables
    inWorkspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\E2FCAStep2Rural.gdb"
    outWorkspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\E2FCAStep2Rural.gdb"
    inFeatures = os.path.join(inWorkspace,"CentroidCatchHos3")
    outFeatures = os.path.join(outWorkspace,"CentroidCatchHos3_Diss")
    dissolveFields = ["FacilityID"]
    statFields = [["Hos3E2FCASlow","sum"],["Hos3E2FCAQuick","sum"]]
    
    # Execute Dissolve using FaclityID as Dissolve Fields, summing the weighted PropToPop(Slow & Quick) of 3 time breaks into 1
    arcpy.Dissolve_management(inFeatures, outFeatures, dissolveFields, statFields)
    
    print("Script completed successfully")
    
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


# Join all three tables with Census polygon layer

### PHC

In [39]:
import arcpy
from arcpy import env
import os
try:
    # Set environment settings
    env.workspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\E2FCAStep2Rural.gdb"
    
    # Set the local parameters
    inWorkspace1 = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\HealthPlaceNew.gdb"
    inWorkspace2 = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\E2FCAStep2Rural.gdb"
    inFeatures = os.path.join(inWorkspace1,"Census2015Final")
    joinField1 = "OBJECTID"
    joinTable = os.path.join(inWorkspace2,"CentroidCatchPHC_Diss")
    joinField2 = "FacilityID"
    fieldList = ["SUM_PHCE2FCASlow", "SUM_PHCE2FCAQuick"] 
    outFeatures = os.path.join(inWorkspace2,"CensusE2FCAPHC")
    
    # Join two feature classes by the zonecode field and only carry 
    # over the PropToPopSlow and PropToPopQuick fields
    arcpy.JoinField_management(inFeatures, joinField1, joinTable, joinField2, fieldList)
    arcpy.CopyFeatures_management(inFeatures, outFeatures)

    print("Script completed successfully")
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


### Hos2

In [42]:
import arcpy
from arcpy import env
import os
try:
    # Set environment settings
    env.workspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\E2FCAStep2Rural.gdb"
    
    # Set the local parameters
    inWorkspace1 = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\HealthPlaceNew.gdb"
    inWorkspace2 = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\E2FCAStep2Rural.gdb"
    inFeatures = os.path.join(inWorkspace2,"CensusE2FCAPHC")
    joinField1 = "OBJECTID"
    joinTable = os.path.join(inWorkspace2,"CentroidCatchHos2_Diss")
    joinField2 = "FacilityID"
    fieldList = ["SUM_Hos2E2FCASlow", "SUM_Hos2E2FCAQuick"] 
    outFeatures = os.path.join(inWorkspace2,"CensusE2FCAPHCHos2")
    
    # Join two feature classes by the zonecode field and only carry 
    # over the PropToPopSlow and PropToPopQuick fields
    arcpy.JoinField_management(inFeatures, joinField1, joinTable, joinField2, fieldList)
    arcpy.CopyFeatures_management(inFeatures, outFeatures)
 
    print("Script completed successfully")
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


### Hos3

In [44]:
import arcpy
from arcpy import env
import os
try:
    # Set environment settings
    env.workspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\E2FCAStep2Rural.gdb"
    
    # Set the local parameters
    inWorkspace1 = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\HealthPlaceNew.gdb"
    inWorkspace2 = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\E2FCAStep2Rural.gdb"
    inFeatures = os.path.join(inWorkspace2,"CensusE2FCAPHCHos2")
    joinField1 = "OBJECTID"
    joinTable = os.path.join(inWorkspace2,"CentroidCatchHos3_Diss")
    joinField2 = "FacilityID"
    fieldList = ["SUM_Hos3E2FCASlow", "SUM_Hos3E2FCAQuick"] 
    outFeatures = os.path.join(inWorkspace2,"CensusE2FCAPHCHos2Hos3")

    # Join two feature classes by the zonecode field and only carry 
    # over the PropToPopSlow and PropToPopQuick fields
    arcpy.JoinField_management(inFeatures, joinField1, joinTable, joinField2, fieldList)
    arcpy.CopyFeatures_management(inFeatures, outFeatures)

    print("Script completed successfully")
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


# Finally, export the feature and create a final feature - CensusE2FCAFinal

## Additional data - Bus stop density

### Spatial join with bus stop point feature 

In [46]:
### Import system modules
import arcpy
import os

try:
    
    # Set local variables
    inWorkspace1 = r"C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/E2FCAStep2Rural.gdb"
    inWorkspace2 = r"C:\Users\kenta\Documents\HDDFiles\GISData_Import\Esri詳細2015中国四国\MS_SCALE.gdb"
    outWorkspace = r"C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/E2FCAStep2Rural.gdb"
      
    # Want to join PHCCover polygon to KIHON_Chugoku2010 and calculate
    # the total number of population that service area (10, 20, 30 min) can cover
    targetFeatures = os.path.join(inWorkspace1, "CensusE2FCAPHCHos2Hos3")
    joinFeatures = os.path.join(inWorkspace2, "POI_BUSSTOP")
    
    # Output will be the target features: PHCCover Polygon with a sum of population field 
    outfc = os.path.join(outWorkspace, "CensusE2FCAFinal")
    
    # Create a new fieldmappings and add the two input feature classes.
    fieldmappings = arcpy.FieldMappings()
    fieldmappings.addTable(targetFeatures)
    fieldmappings.addTable(joinFeatures)
        
    # Delete fields that are no longer applicable, such as city CITY_NAME and CITY_FIPS
    # as only the first value will be used by default
    x = fieldmappings.findFieldMapIndex("BSN")
    fieldmappings.removeFieldMap(x)
    
    #Run the Spatial Join tool, using the defaults for the join operation and join type
    arcpy.SpatialJoin_analysis(targetFeatures, joinFeatures, outfc, "JOIN_ONE_TO_ONE", "KEEP_ALL", fieldmappings,"COMPLETELY_CONTAINS")
 
    print("Script completed successfully")
    
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


# Missing imputation of E2FCA score

### Several units are missing E2FCA score due to the network construction aka systematic issue
### Imputing a mean score of neighboring units (n=8)
### Imputing 0 for the units which are clearly out of catchments

### First, impute mean for Hos2 and Hos3 by using conditinal statements (if SUM_PHCE2FCASlow = None)

In [ ]:
### from pandas.core.reshape import (pivot_simple as pivot, get_dummies,
### ImportError: cannot import name 'pivot_simple'
### This didnt work...
############ Need to impute missing value with different PC manually############

In [8]:
import arcpy
from arcpy import env
try:
    env.workspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\E2FCAStep2Rural.gdb"
    env.overwriteOutput = True
    
    inFeatures = "CensusE2FCAFinal"
    outFeatures = "CensusE2FCAFinalMI"
    fields_to_fill1 = "SUM_Hos2E2FCASlow"
    fields_to_fill2 = "SUM_Hos2E2FCAQuick"
    fields_to_fill3 = "SUM_Hos3E2FCASlow"
    fields_to_fill4 = "SUM_Hos3E2FCAQuick"
    fill_method = "AVERAGE"
    conceptualization_of_spatial_relationships = "CONTIGUITY_EDGES_ONLY"
    
    arcpy.FillMissingValues_stpm(inFeatures, outFeatures, fields_to_fill1, fill_method, conceptualization_of_spatial_relationships)
    arcpy.FillMissingValues_stpm(outFeatures, outFeatures, fields_to_fill2, fill_method, conceptualization_of_spatial_relationships)
    arcpy.FillMissingValues_stpm(outFeatures, outFeatures, fields_to_fill3, fill_method, conceptualization_of_spatial_relationships)
    arcpy.FillMissingValues_stpm(outFeatures, outFeatures, fields_to_fill4, fill_method, conceptualization_of_spatial_relationships)

    # Write the selected features to a new featureclass
    arcpy.CopyFeatures_management(outFeatures, outFeatures)

    print("Script completed successfully")

except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))
    

An error occurred on line 19

Traceback (most recent call last):
  File "<string>", line 3479, in execute
  File "C:\Users\kenta\AppData\Local\Programs\ArcGIS\Pro\Resources\ArcToolbox\Scripts\SSImpute.py", line 19, in <module>
    import SSDataObject as SSDO
  File "C:\Users\kenta\AppData\Local\Programs\ArcGIS\Pro\Resources\ArcToolbox\Scripts\SSDataObject.py", line 23, in <module>
    import SSUtilities as UTILS
  File "C:\Users\kenta\AppData\Local\Programs\ArcGIS\Pro\Resources\ArcToolbox\Scripts\SSUtilities.py", line 26, in <module>
    import WeightsUtilities as WU
  File "C:\Users\kenta\AppData\Local\Programs\ArcGIS\Pro\Resources\ArcToolbox\Scripts\WeightsUtilities.py", line 22, in <module>
    import SSPanelObject as SSPO
  File "C:\Users\kenta\AppData\Local\Programs\ArcGIS\Pro\Resources\ArcToolbox\Scripts\SSPanelObject.py", line 18, in <module>
    import pandas as PANDAS
  File "C:\Users\kenta\AppData\Local\Programs\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\panda

### Second, impute 0 for the rest of units = out of catchments

In [ ]:
### This didnt work...

In [14]:
import arcpy
from arcpy import env
try:
    # Set environment settings
    env.workspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\E2FCAStep2Rural.gdb"
    
    # Set local variables
    inTable = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\E2FCAStep2Rural.gdb\CensusE2FCAFinal"
    fieldName1 = "SUM_Hos2E2FCASlow"
    fieldName2 = "SUM_Hos2E2FCAQuick"
    fieldName3 = "SUM_Hos3E2FCASlow"
    fieldName4 = "SUM_Hos3E2FCAQuick"
    expression1 = "w(!SUM_PHCE2FCASlow!,!SUM_Hos2E2FCASlow!)"
    expression2 = "w(!SUM_PHCE2FCASlow!,!SUM_Hos2E2FCAQuick!)"
    expression3 = "w(!SUM_PHCE2FCASlow!,!SUM_Hos3E2FCASlow!)"
    expression4 = "w(!SUM_PHCE2FCASlow!,!SUM_Hos3E2FCAQuick!)"
    codeblock1 = """def w (f0,f1):
        if (f0 != None and f1 is None):
            return 0"""
    codeblock2 = """def w (f0,f2):
        if f0 is None:
            return f2
        elif (f0 is not None and f2 is None):
            return 0"""
    codeblock3 = """def w (f0,f3):
        if f0 is None:
            return f3
        elif (f0 is not None and f3 is None):
            return 0"""
    codeblock4 = """def w (f0,f4):
        if f0 is None:
            return f4
        elif (f0 is not None and f4 is None):
            return 0"""
    # Execute CalculateField - imputing 0 where PHC score is not equal to NULL
    arcpy.CalculateField_management(inTable, fieldName1, expression1, "PYTHON_9.3", codeblock1)
    arcpy.CalculateField_management(inTable, fieldName2, expression2, "PYTHON_9.3", codeblock2)
    arcpy.CalculateField_management(inTable, fieldName3, expression3, "PYTHON_9.3", codeblock3)
    arcpy.CalculateField_management(inTable, fieldName4, expression4, "PYTHON_9.3", codeblock4)


    print("Script completed successfully")

except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


### Calculate bus stop density

In [48]:
### Import system modules
import arcpy
import os

try:
    
    # Create a new field of Bus stop count and Bus stop density
    env.workspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\E2FCAStep2Rural.gdb"
    inTable = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\E2FCAStep2Rural.gdb\CensusE2FCAFinal"
    fieldName1 = "BusCount"
    fieldName2 = "BusDen"
    expression1 = "!Join_Count!"
    expression2 = "!BusCount!/!AreaRes!"
    dropFields = "JOIN_COUNT"
    # Execute AddField
    arcpy.AddField_management(inTable, fieldName1, "DOUBLE")
    arcpy.AddField_management(inTable, fieldName2, "DOUBLE")
    # Execute CalculateField - weighting
    arcpy.CalculateField_management(inTable, fieldName1, expression1, "PYTHON_9.3")
    arcpy.CalculateField_management(inTable, fieldName2, expression2, "PYTHON_9.3")
    # Delete JOIN_COUNT field
    arcpy.DeleteField_management(inTable, dropFields)
    
    print("Script completed successfully")
    
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


### Manually finished missing imputation by different PC
### Now make a final dataset
### 1. Copy imputed data to a project folder
### 2. Join imputed data's fields to CensusE2FCAFinal data and create a new feature "CensusE2FCAFinalMI"
### 3. Delete duplicated E2FCA score fields
### 4. Spatial join with dfnew Gecoded

In [ ]:
### 1. Copy imputed data to a project folder

In [1]:
import arcpy
from arcpy import env
try:
    
    env.workspace = r"G:\ArcGIS\Projects\HealthPlaceNew\E2FCAStep2Rural.gdb"
    ### Set a path for created data will ve stored
    outpath = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\E2FCAStep2Rural.gdb"
    # Export data to new path
    arcpy.FeatureClassToFeatureClass_conversion('CensusE2FCAMI',outpath,"CensusE2FCAMI")
    
    print("Script completed successfully")
    
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))    

In [ ]:
# This works!

In [ ]:
### 2. Join imputed data's fields to CensusE2FCAFinal data and create a new feature "CensusE2FCAFinalMI"

In [6]:
import arcpy
from arcpy import env
import os
try:
    # Set environment settings
    env.workspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\E2FCAStep2Rural.gdb"
    
    # Set the local parameters
    inWorkspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\E2FCAStep2Rural.gdb"
    outWorkspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\E2FCAStep2Rural.gdb"
    inFeatures = os.path.join(inWorkspace,"CensusE2FCAFinal")
    joinField = "OBJECTID"
    joinTable = os.path.join(inWorkspace,"CensusE2FCAMI")
    fieldList = ["SUM_PHCE2FCASLOW", "SUM_PHCE2FCAQUICK", "SUM_HOS2E2FCASLOW", "SUM_HOS2E2FCAQUICK", "SUM_HOS3E2FCASLOW", "SUM_HOS3E2FCAQUICK"] 
    outFeatures = os.path.join(outWorkspace,"CensusE2FCAFinalMI")
    
    # Join two feature classes by the zonecode field and only carry 
    # over the PropToPopSlow and PropToPopQuick fields
    arcpy.JoinField_management(inFeatures, joinField, joinTable, joinField, fieldList)
    arcpy.CopyFeatures_management(inFeatures, outFeatures)

    print("Script completed successfully")
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


In [ ]:
### 3. Delete duplicated E2FCA score fields

In [8]:
import arcpy
from arcpy import env
try:
    env.workspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\E2FCAStep2Rural.gdb"
    # list all field name
    field_names = [f.name for f in arcpy.ListFields("CensusE2FCAFinalMI")]
    print(field_names)
    
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

['OBJECTID', 'Shape', 'TARGET_FID', 'KEN_NAME', 'SHICHO_NAME', 'SHI_NAME', 'KU_NAME', 'GUN_NAME', 'CHOSON_NAME', 'OAZA_NAME', 'KOAZA_NAME', 'SUM_AREA', 'TOTPOP_H27', 'AreaRes', 'PopDen', 'SUM_PHCE2FCASlow', 'SUM_PHCE2FCAQuick', 'SUM_Hos2E2FCASlow', 'SUM_Hos2E2FCAQuick', 'SUM_Hos3E2FCASlow', 'SUM_Hos3E2FCAQuick', 'BusCount', 'BusDen', 'SUM_PHCE2FCASLOW_1', 'SUM_PHCE2FCAQUICK_1', 'SUM_HOS2E2FCASLOW_1', 'SUM_HOS2E2FCAQUICK_1', 'SUM_HOS3E2FCASLOW_1', 'SUM_HOS3E2FCAQUICK_1', 'Shape_Length', 'Shape_Area']


In [9]:
import arcpy
from arcpy import env
try:    
    env.workspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\E2FCAStep2Rural.gdb"
    # Choose the fields want to be deleted
    
    delfield = ['SUM_PHCE2FCASlow', 'SUM_PHCE2FCAQuick', 'SUM_Hos2E2FCASlow', 'SUM_Hos2E2FCAQuick', 'SUM_Hos3E2FCASlow', 'SUM_Hos3E2FCAQuick']   # Delete fields
    arcpy.DeleteField_management('CensusE2FCAFinalMI',delfield)
    
    print("Script completed successfully")
    
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

In [ ]:
# Well done!!

### Found the area which has innappropriate population density
### Replace manually - if OAZA_AZA = "津和野町富田": return 498.34528

In [13]:
import arcpy
from arcpy import env
try:
    # Set environment settings
    env.workspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\E2FCAStep2Rural.gdb"
    
    # Set local variables
    inTable = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\E2FCAStep2Rural.gdb\CensusE2FCAFinalMI"
    fieldName1 = "PopDen"
    expression1 = "w(!PopDen!)"
    codeblock1 = """def w (f1):
        if f1 > 50000:
            return 498.34528
        else:
            return f1"""
    # Execute CalculateField - imputing 0 where PHC score is not equal to NULL
    arcpy.CalculateField_management(inTable, fieldName1, expression1, "PYTHON_9.3", codeblock1)


    print("Script completed successfully")

except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


# Spatial Join with geocoded Kosha2015 data

In [14]:
### Import system modules
import arcpy
import os

try:
    
    # Set local variables
    inWorkspace1 = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\E2FCA.gdb"
    inWorkspace2 = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\E2FCAStep2Rural.gdb"
    outWorkdspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\HealthPlaceNew.gdb"
    targetFeatures = os.path.join(inWorkspace1, "dfnew6Gecoded")
    joinFeatures = os.path.join(inWorkspace2, "CensusE2FCAFinalMI")
    
    # Output will be the target features: PHCCover Polygon with a sum of population field 
    outfc = os.path.join(outWorkdspace, "dfnew6E2CAFinal")
    
    # Create a new fieldmappings and add the two input feature classes.
    fieldmappings = arcpy.FieldMappings()
    fieldmappings.addTable(targetFeatures)
    fieldmappings.addTable(joinFeatures)
       
    # Delete fields that are no longer applicable, such as city CITY_NAME and CITY_FIPS
    # as only the first value will be used by default
    x = fieldmappings.findFieldMapIndex("Status")
    fieldmappings.removeFieldMap(x)
    y = fieldmappings.findFieldMapIndex("Score")
    fieldmappings.removeFieldMap(y)
    z = fieldmappings.findFieldMapIndex("Match_type")
    fieldmappings.removeFieldMap(z)
    b = fieldmappings.findFieldMapIndex("Year")
    fieldmappings.removeFieldMap(b)
    c = fieldmappings.findFieldMapIndex("Pref")
    fieldmappings.removeFieldMap(c)
    d = fieldmappings.findFieldMapIndex("Gun")
    fieldmappings.removeFieldMap(d)
    e = fieldmappings.findFieldMapIndex("City")
    fieldmappings.removeFieldMap(e)
    f = fieldmappings.findFieldMapIndex("Ku")
    fieldmappings.removeFieldMap(f)
    h = fieldmappings.findFieldMapIndex("Match_code")
    fieldmappings.removeFieldMap(h)
    i = fieldmappings.findFieldMapIndex("Addr_type")
    fieldmappings.removeFieldMap(i)
    j = fieldmappings.findFieldMapIndex("In_Single_Line_Input")
    fieldmappings.removeFieldMap(j)
    k = fieldmappings.findFieldMapIndex("TARGET_FID")
    fieldmappings.removeFieldMap(k)
    
    #Run the Spatial Join tool, using the defaults for the join operation and join type
    arcpy.SpatialJoin_analysis(targetFeatures, joinFeatures, outfc, "JOIN_ONE_TO_ONE", "KEEP_ALL", fieldmappings,"COMPLETELY_WITHIN")

    print("Script completed successfully")
    
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


# Export data as csv or xls format

### First, create a gdb table from point feature

In [15]:
# Import system modules
import arcpy
from arcpy import env

try:
    # Set environment settings
    env.workspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\HealthPlaceNew.gdb"
    # Set local variables
    inTable = "dfnew6E2CAFinal"
    outLocation = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\HealthPlaceNew.gdb"
    outTable = "dfnew6E2FCAFinalTable"
    
    # Execute TableToTable
    arcpy.TableToTable_conversion(inTable, outLocation, outTable)

    print("Script completed successfully")
    
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


In [ ]:
### Now export the table to xls format file

In [16]:
# Import system modules
import arcpy
from arcpy import env

try:
    # Set environment settings
    env.workspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\HealthPlaceNew.gdb"
    # Set local variables
    inTable = "dfnew6E2FCAFinalTable"
    out_xls = r"G:\dfnew6E2FCAFinal.xls"
    
    arcpy.TableToExcel_conversion(inTable, out_xls)

    print("Script completed successfully")
    
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully
